# PREDICCIÓN DEL ABANDONO (BURNOUT) DE EMPLEADOS

# Importación de librerías

In [34]:
import pickle as pkl

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold

from catboost import CatBoostClassifier

# Importación de datos

In [35]:
data = pkl.load(open("datos_grupos/attrition_available_2.pkl",'rb'))

# Analisis Exploratorio de Datos

Comprobamos la cantidad de entradas que tenemos en el dataset y los atributos presentes en estas.

In [36]:
print("num de instancias y atributos:", data.shape)
print("Nombre de los atributos:", data.columns)

num de instancias y atributos: (4410, 31)
Nombre de los atributos: Index(['hrs', 'absences', 'JobInvolvement', 'PerformanceRating',
       'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'Age',
       'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EmployeeCount', 'EmployeeID', 'Gender',
       'JobLevel', 'JobRole', 'MaritalStatus', 'MonthlyIncome',
       'NumCompaniesWorked', 'Over18', 'PercentSalaryHike', 'StandardHours',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager'],
      dtype='object')


Comprobamos si existen missing values en los datos; para ello utilizamos el método .info()

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4410 entries, 1 to 4409
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   hrs                      3639 non-null   float64
 1   absences                 3575 non-null   float64
 2   JobInvolvement           3585 non-null   float64
 3   PerformanceRating        3534 non-null   float64
 4   EnvironmentSatisfaction  3428 non-null   float64
 5   JobSatisfaction          3637 non-null   float64
 6   WorkLifeBalance          3620 non-null   float64
 7   Age                      3636 non-null   float64
 8   Attrition                4410 non-null   object 
 9   BusinessTravel           3687 non-null   object 
 10  Department               3575 non-null   object 
 11  DistanceFromHome         3681 non-null   float64
 12  Education                3628 non-null   float64
 13  EducationField           4410 non-null   object 
 14  EmployeeCount           

Observamos missing values en la mayoria de atributos.
La variable de salida se corresponde con el índice 8 (Attrition) 

Observamos cada atributo con más detalle; buscando su tipo, si es constante y la proporción de missing values que presenta

In [38]:
info = {}
for colum in data:
    print(data[colum].value_counts(dropna = False, normalize = True ).to_frame())
    info[colum] = data[colum].value_counts(dropna = False, normalize = True ).to_frame()

               hrs
NaN       0.174830
6.033902  0.000454
9.853332  0.000454
6.002747  0.000227
5.691867  0.000227
...            ...
6.511941  0.000227
8.338820  0.000227
6.623272  0.000227
6.884605  0.000227
6.511790  0.000227

[3638 rows x 1 columns]
      absences
NaN   0.189342
7.0   0.049206
17.0  0.047846
6.0   0.045805
14.0  0.045578
19.0  0.045578
10.0  0.044671
8.0   0.044218
11.0  0.043991
18.0  0.043991
15.0  0.043991
12.0  0.043311
16.0  0.041270
13.0  0.040590
9.0   0.039683
20.0  0.039229
5.0   0.038095
21.0  0.032653
4.0   0.026304
22.0  0.020635
3.0   0.013605
23.0  0.011111
2.0   0.006576
24.0  0.001587
1.0   0.001134
     JobInvolvement
3.0        0.478912
2.0        0.208163
NaN        0.187075
4.0        0.078912
1.0        0.046939
     PerformanceRating
3.0           0.678458
NaN           0.198639
4.0           0.122902
     EnvironmentSatisfaction
3.0                 0.237415
4.0                 0.236508
NaN                 0.222676
2.0                 0.153515


Consideramos que si una columna solo tiene dos valores (un valor y NaN) esta es constante; en este caso las columnas de Over18 y Standard Hours. Ignoramos la columna de Attrition ya que es nuestra variable objetivo y se trata de una clasificación binaria.

In [39]:
for colum in data:
    if info[colum].shape[0] < 3:
        print(info[colum])


     Attrition
No    0.838776
Yes   0.161224
     EmployeeCount
1.0       0.790023
NaN       0.209977
       Over18
Y    0.805215
NaN  0.194785
     StandardHours
8.0        0.79161
NaN        0.20839


In [40]:
data['Attrition'] = data['Attrition'].map({'Yes': 1, 'No': 0})

Como ya se ha mencionado se trata de un problema de clasificación binaria. Y está desbalanceado (un 83% de la variable de salida se corresponde con una de las clases) 

# Preparación de datos

## Train y test

Dividimos nuestro dataset en X e y, ieendo X los atributos y Y la variable de salida. De la misma manera, creamos dos listas para guardar los datos categóricos y numéricos. Además, dividimos los datos en train y test.

In [41]:
X = data.drop("Attrition", axis= 'columns')
Y = data['Attrition']

cat_col = []
num_col = []

for col in X.columns:
    if X[col].dtype != "object":
        num_col.append(col)
        continue
    cat_col.append(col)


train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)



## Pipeline

Preparamos el preproceso de datos, los valores numéricos serán escalados y sus missing values se imputarán de acuerdo a la mediana. Para los categóricos se aplicará una transformación usando one-hot-encoding y sus missing values se imputarán de acuerdo al más frecuente. Estas dos transformaciones se combinan en el pipeline procesor para ser utilizado más adelante con los diferentes modelos.

In [42]:
imputer_num = SimpleImputer(strategy='median')
scaler = RobustScaler()
pipeline_num = Pipeline(
    steps=[
        ("imputer", imputer_num),
        ("scaler", scaler)
    ]
)

imputer_cat = SimpleImputer(strategy='most_frequent')
encoder_cat = OneHotEncoder(handle_unknown='ignore')
pipeline_cat = Pipeline(
    steps=[
        ("imputer", imputer_cat),
        ("encoder", encoder_cat)
    ]
)

processor = ColumnTransformer(
    transformers=[
        ("num", pipeline_num, num_col),
        ("cat", pipeline_cat, cat_col),
    ]
)

transformed_x = processor.fit_transform(train_x)
print(transformed_x.shape)

(3528, 50)


# Modelos

## Logistic Regression

In [43]:
Log_reg = LogisticRegression(class_weight='balanced', random_state=2)

predictor = Pipeline(
    steps=[
        ("Transformer", processor),
        ("predictor", Log_reg)
    ]
)

In [44]:
predictor.fit(train_x, train_y)

pred = predictor.predict(test_x)

print("Clasification report: \n\n",classification_report(test_y, pred, zero_division=0))
print( "\nMatriz de confusión: \n\n", confusion_matrix(test_y, pred))
print("\nScore f1:\n", f1_score(test_y, pred, zero_division=0))



Clasification report: 

               precision    recall  f1-score   support

           0       0.91      0.74      0.82       740
           1       0.31      0.62      0.42       142

    accuracy                           0.72       882
   macro avg       0.61      0.68      0.62       882
weighted avg       0.81      0.72      0.75       882


Matriz de confusión: 

 [[547 193]
 [ 54  88]]
Score f1:
 0.4160756501182033


## Boosting

In [45]:
gb_clas = GradientBoostingClassifier(random_state=2)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.05, 0.01]
}

cv = StratifiedKFold(n_splits=5)

grid = GridSearchCV(gb_clas, param_grid, cv=cv, scoring='balanced_accuracy', n_jobs=-1)

predictor = Pipeline(
    steps=[
        ("Transformer", processor),
        ("predictor", grid)
    ]
)

In [46]:
predictor.fit(train_x, train_y)
pred = predictor.predict(test_x)

print("Clasification report: \n\n",classification_report(test_y, pred, zero_division=0))
print("\nMatriz de confusión: \n\n", confusion_matrix(test_y, pred))
print("\nScore f1:\n", f1_score(test_y, pred, zero_division=0))



Clasification report: 

               precision    recall  f1-score   support

           0       0.92      0.99      0.95       740
           1       0.93      0.55      0.69       142

    accuracy                           0.92       882
   macro avg       0.92      0.77      0.82       882
weighted avg       0.92      0.92      0.91       882


Matriz de confusión: 

 [[734   6]
 [ 64  78]]
Score f1:
 0.6902654867256638


## Catboost

In [47]:
catboost = CatBoostClassifier(random_seed=2, verbose=0, class_weights= [1, 5.2])

param_grid = { 'iterations': [100, 200, 300],
    'l2_leaf_reg': [1, 3, 5],
    'depth': [4, 6, 8],
    'border_count': [32, 64, 128]  
}

cat_grid = GridSearchCV(catboost, param_grid, cv=cv, scoring='balanced_accuracy', n_jobs=-1)

cat_predictor = Pipeline(
    steps=[
        ("Transformer", processor),
        ("predictor", cat_grid)
    ]
)   

In [48]:
cat_predictor.fit(train_x, train_y)
pred = cat_predictor.predict(test_x)

print("Clasification report: \n\n",classification_report(test_y, pred, zero_division=0))
print("\nMatriz de confusión: \n\n", confusion_matrix(test_y, pred))
print("\nScore f1:\n", f1_score(test_y, pred, zero_division=0))

Clasification report: 

               precision    recall  f1-score   support

           0       0.93      0.94      0.94       740
           1       0.67      0.65      0.66       142

    accuracy                           0.89       882
   macro avg       0.80      0.80      0.80       882
weighted avg       0.89      0.89      0.89       882


Matriz de confusión: 

 [[695  45]
 [ 49  93]]
Score f1:
 0.6642857142857143


## Conclusiones

Habiendo entrenado y evaluado todos los modelos, podemos observar que el regresor logístico presenta el mayor error. Por lo que lo podemos descartar como la opción óptima. Por otra parte los modelos de gradientboost y catboosting presentan un error similar, siendo el gradientboost el que presenta una fq_score algo mayor. Por lo que podemos concluir que el modelo de gradientboost es el que mejor se ajusta a nuestro problema.

## SelectKBest

Una vez evaluados los tres modelos; elegimos el mejor de estos (gradientboost) para probar si reduciendo el número de atributos obtenemos mejores resultados.

In [49]:
selectk = SelectKBest(score_func=f_classif, k=40)

Selector = Pipeline(
    steps=[
        ("preprocessor", processor),
        ("variance_threshold", VarianceThreshold()),
        ("kbest", selectk),
        ("predictor", GradientBoostingClassifier(**grid.best_params_, random_seed= 2, verbose=0))
    ]
)

X_best = Selector.fit(train_x,train_y)
pred = X_best.predict(test_x)

print("Clasification report: \n\n",classification_report(test_y, pred, zero_division=0))
print("\nMatriz de confusión: \n\n", confusion_matrix(test_y, pred))
print("\nScore f1:\n", f1_score(test_y, pred, zero_division=0))

Clasification report: 

               precision    recall  f1-score   support

           0       0.94      0.94      0.94       740
           1       0.69      0.68      0.68       142

    accuracy                           0.90       882
   macro avg       0.81      0.81      0.81       882
weighted avg       0.90      0.90      0.90       882


Matriz de confusión: 

 [[696  44]
 [ 46  96]]

Score f1:
 0.6808510638297872


## Conclusiones

Aunque la diferencia no sea muy elevada, podemos observar una mejoría del rendimiento. Quizás el conjunto de test sea demasiado pequeño como para afirmar con certeza que aplicar la reducción de atributos es positiva; pero nosotros nos decantamos de forma favorable hacia esta.